In [143]:
import pandas as pd
import random
import os
import math
import numpy as np
from sklearn import datasets, tree
from sklearn.model_selection import train_test_split
from sklearn import  metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [144]:
class CFG:
    SEED = 42

In [145]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

In [146]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
add = pd.read_csv('./snp_info.csv')


from sklearn.preprocessing import MinMaxScaler





def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x

train_x, train_y = get_x_y(train)
test_x = get_x_y(test)

add = add.drop(columns=['SNP_id', 'name'])

from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std.fit(add)
X_train_scaled = std.transform(add)




In [147]:
class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]

snp_data = []
for col in snp_col:
    snp_data += list(train_x[col].values)

train_y = class_le.fit_transform(train_y)
snp_le.fit(snp_data)

LabelEncoder()

In [163]:
for col in train_x.columns:
    if col in snp_col:
        train_x[col] = snp_le.transform(train_x[col])
        test_x[col] = snp_le.transform(test_x[col])

C:\ProgramData\Anaconda3\envs\tf_cpu\lib\site-packages\numpy\lib\arraysetops.py:604: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [256]:
from xgboost import XGBClassifier

xgb_wrapper = XGBClassifier(n_estimators=800, learning_rate=0.001, max_depth=5) ## 400, 0.001, 3
xgb_wrapper.fit(train_x, train_y)




clf = RandomForestClassifier(n_estimators=400, random_state=CFG.SEED)
clf.fit(train_x, train_y)
clf_predicted = clf.predict(test_x)

#KNN
knn = KNeighborsClassifier(n_neighbors=262).fit(train_x, train_y)
knn_predicted = knn.predict(test_x)

#SVM
svm = SVC(C=0.1, gamma=0.003, probability=True, random_state=CFG.SEED).fit(train_x, train_y)
svm_predicted = svm.predict(test_x)

#dtree
dtree = tree.DecisionTreeClassifier(criterion='gini', max_depth=7,  random_state=CFG.SEED , min_samples_split=10, min_samples_leaf=2)
dtree = dtree.fit(train_x, train_y)
dtree_predicted = dtree.predict(test_x)


C:\ProgramData\Anaconda3\envs\tf_cpu\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:20:13] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [257]:
preds = xgb_wrapper.predict(test_x)
print('Done.')
print(preds)

metrics.accuracy_score(preds, clf_predicted)

Done.
[0 1 2 1 0 2 2 1 0 0 2 1 2 0 1 1 0 1 1 2 1 1 1 0 1 1 1 0 0 1 0 0 1 1 0 1 2
 1 1 2 0 1 2 1 1 1 1 2 1 2 0 1 0 1 1 1 2 0 1 2 0 1 2 2 2 0 1 0 0 1 1 1 0 0
 2 1 2 1 1 1 2 1 0 1 1 1 1 1 2 0 1 1 2 1 1 2 0 1 0 2 0 1 1 2 0 0 2 1 0 1 2
 1 1 1 1 0 0 2 1 1 0 1 1 2 2 1 2 1 0 1 0 0 1 1 1 2 0 0 1 0 0 0 2 1 1 1 0 1
 2 0 0 1 0 1 1 0 0 1 2 0 1 2 2 1 0 0 2 1 1 0 1 2 2 1 1]


0.9828571428571429

In [258]:
submit = pd.read_csv('./sample_submission.csv')

submit['class'] = class_le.inverse_transform(preds)

submit.to_csv('./answer/XGB_submit2.csv', index=False)

In [259]:
p1 = pd.read_csv('./answer/XGB_submit2.csv')
p2 = pd.read_csv('./answer/XGB_submit2.csv')

metrics.accuracy_score(p1, p2)

ValueError: multiclass-multioutput is not supported